# Crosstab and contingency

In [1]:
import pandas as pd

# Create the dataframe
data = {
    "x": ["p", "p", "p", "q", "p", "q", "q", "q"],
    "y": ["r", "r", "r", "s", "s", "r", "s", "r"],
    "Class": ["c1", "c1", "c1", "c1", "c2", "c2", "c2", "c"]
}

df = pd.DataFrame(data)
df

,x,y,Class
0,p,r,c1
1,p,r,c1
2,p,r,c1
3,q,s,c1
4,p,s,c2
5,q,r,c2
6,q,s,c2
7,q,r,c


In [5]:
df.value_counts()

x  y  Class
p  r  c1       3
   s  c2       1
q  r  c        1
      c2       1
   s  c1       1
      c2       1
Name: count, dtype: int64

In [6]:
# Create contingency table: counts of x,y combinations vs Class
contingency_table = pd.crosstab(index=[df["x"], df["y"]], columns=df["Class"])
contingency_table

Class  c  c1  c2
x y             
p r    0   3   0
  s    0   0   1
q r    1   0   1
  s    0   1   1

# Dashboard Example

In [5]:
"""
Conversion Rate & User Activity Dashboard
Compatible with Databricks environment and local Python

To run locally: pip install dash plotly pandas numpy
To run in Databricks: Use %pip install dash plotly in a notebook cell
"""

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, dcc, html, Input, Output, callback
from plotly.subplots import make_subplots

# ============================================================================
# 1. DATA GENERATION (In production, this would be replaced with Delta Table reads)
# ============================================================================

def generate_user_events():
    """
    Simulate user event data that would typically come from your data lake.
    
    In Databricks, you would replace this with:
    spark.read.table("events.user_activity") or
    pd.read_sql("SELECT * FROM conversion_funnel", connection)
    """
    np.random.seed(42)
    
    # Date range for historical data
    start_date = datetime(2024, 1, 1)
    dates = [start_date + timedelta(days=x) for x in range(90)]
    
    events_data = []
    user_id = 1
    
    for date in dates:
        # Simulate daily new users with some variability
        daily_users = np.random.randint(100, 200)
        
        for _ in range(daily_users):
            # User journey through funnel
            user_events = {
                'user_id': f'U{user_id:06d}',
                'signup_date': date,
                'channel': np.random.choice(['organic', 'paid_search', 'social', 'referral'], 
                                           p=[0.3, 0.35, 0.2, 0.15]),
                'device': np.random.choice(['mobile', 'desktop', 'tablet'], 
                                          p=[0.6, 0.35, 0.05]),
                'country': np.random.choice(['US', 'UK', 'DE', 'FR', 'CA'], 
                                           p=[0.4, 0.2, 0.15, 0.15, 0.1])
            }
            
            # Funnel stages with conversion probabilities
            visited_homepage = True
            user_events['stage_homepage'] = visited_homepage
            
            # Stage 2: Product View (80% conversion from homepage)
            viewed_product = np.random.random() < 0.80
            user_events['stage_product_view'] = viewed_product
            
            # Stage 3: Add to Cart (40% conversion from product view)
            if viewed_product:
                added_to_cart = np.random.random() < 0.40
                user_events['stage_add_to_cart'] = added_to_cart
            else:
                added_to_cart = False
                user_events['stage_add_to_cart'] = False
            
            # Stage 4: Checkout (60% conversion from cart)
            if added_to_cart:
                started_checkout = np.random.random() < 0.60
                user_events['stage_checkout'] = started_checkout
            else:
                started_checkout = False
                user_events['stage_checkout'] = False
            
            # Stage 5: Purchase (70% conversion from checkout)
            if started_checkout:
                completed_purchase = np.random.random() < 0.70
                user_events['stage_purchase'] = completed_purchase
                if completed_purchase:
                    user_events['purchase_value'] = np.random.uniform(20, 200)
            else:
                completed_purchase = False
                user_events['stage_purchase'] = False
                user_events['purchase_value'] = 0
            
            # Activity metrics
            user_events['sessions_count'] = np.random.poisson(3) + 1
            user_events['total_time_minutes'] = np.random.exponential(15) * user_events['sessions_count']
            
            events_data.append(user_events)
            user_id += 1
    
    return pd.DataFrame(events_data)


def calculate_conversion_metrics(df):
    """
    Calculate conversion funnel metrics.
    In production, this could be a scheduled Databricks job writing to a metrics table.
    """
    funnel_stages = ['stage_homepage', 'stage_product_view', 'stage_add_to_cart', 
                     'stage_checkout', 'stage_purchase']
    
    # Overall conversion rates
    conversion_summary = []
    total_users = len(df)
    
    for i, stage in enumerate(funnel_stages):
        users_at_stage = df[stage].sum()
        conversion_rate = (users_at_stage / total_users) * 100
        
        if i > 0:
            previous_stage = funnel_stages[i-1]
            users_at_previous = df[previous_stage].sum()
            step_conversion = (users_at_stage / users_at_previous) * 100 if users_at_previous > 0 else 0
        else:
            step_conversion = 100
        
        conversion_summary.append({
            'stage': stage.replace('stage_', '').replace('_', ' ').title(),
            'users': int(users_at_stage),
            'overall_conversion_rate': conversion_rate,
            'step_conversion_rate': step_conversion
        })
    
    return pd.DataFrame(conversion_summary)


def calculate_daily_metrics(df):
    """Calculate daily aggregated metrics for time series charts"""
    df['signup_date'] = pd.to_datetime(df['signup_date'])
    
    daily_metrics = df.groupby('signup_date').agg({
        'user_id': 'count',
        'stage_purchase': 'sum',
        'purchase_value': 'sum',
        'sessions_count': 'mean',
        'total_time_minutes': 'mean'
    }).reset_index()
    
    daily_metrics.columns = ['date', 'signups', 'purchases', 'revenue', 
                             'avg_sessions', 'avg_time_minutes']
    daily_metrics['conversion_rate'] = (daily_metrics['purchases'] / daily_metrics['signups']) * 100
    daily_metrics['7day_avg_conversion'] = daily_metrics['conversion_rate'].rolling(7).mean()
    
    return daily_metrics


def calculate_segment_performance(df):
    """Calculate conversion rates by channel and device"""
    # By channel
    channel_metrics = df.groupby('channel').agg({
        'user_id': 'count',
        'stage_purchase': 'sum',
        'purchase_value': 'sum'
    }).reset_index()
    channel_metrics['conversion_rate'] = (channel_metrics['stage_purchase'] / 
                                          channel_metrics['user_id']) * 100
    channel_metrics.columns = ['channel', 'users', 'purchases', 'revenue', 'conversion_rate']
    
    # By device
    device_metrics = df.groupby('device').agg({
        'user_id': 'count',
        'stage_purchase': 'sum'
    }).reset_index()
    device_metrics['conversion_rate'] = (device_metrics['stage_purchase'] / 
                                         device_metrics['user_id']) * 100
    device_metrics.columns = ['device', 'users', 'purchases', 'conversion_rate']
    
    return channel_metrics, device_metrics


# ============================================================================
# 2. LOAD AND PREPARE DATA
# ============================================================================

print("📊 Generating sample data...")
df_events = generate_user_events()
df_conversion = calculate_conversion_metrics(df_events)
df_daily = calculate_daily_metrics(df_events)
df_channel, df_device = calculate_segment_performance(df_events)

print(f"✓ Loaded {len(df_events):,} user records")
print(f"✓ Date range: {df_events['signup_date'].min()} to {df_events['signup_date'].max()}")


# ============================================================================
# 3. BUILD DASHBOARD COMPONENTS
# ============================================================================

def create_kpi_card(title, value, subtitle="", color="#1f77b4"):
    """Create a KPI card component"""
    return html.Div([
        html.Div([
            html.H4(title, style={'margin': '0', 'color': '#666', 'fontSize': '14px'}),
            html.H2(value, style={'margin': '10px 0', 'color': color, 'fontSize': '32px', 'fontWeight': 'bold'}),
            html.P(subtitle, style={'margin': '0', 'color': '#999', 'fontSize': '12px'})
        ], style={'padding': '20px', 'backgroundColor': '#f8f9fa', 'borderRadius': '8px', 
                  'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'})
    ], style={'flex': '1', 'minWidth': '200px', 'margin': '10px'})


def create_funnel_chart(df_conversion):
    """Create conversion funnel visualization"""
    fig = go.Figure()
    
    fig.add_trace(go.Funnel(
        y=df_conversion['stage'],
        x=df_conversion['users'],
        textposition="inside",
        textinfo="value+percent initial",
        marker=dict(
            color=['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A']
        ),
        connector=dict(line=dict(color="royalblue", width=2))
    ))
    
    fig.update_layout(
        title="Conversion Funnel - Overall",
        height=400,
        margin=dict(l=20, r=20, t=40, b=20)
    )
    
    return fig


def create_conversion_trend_chart(df_daily):
    """Create conversion rate over time chart"""
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Conversion rate
    fig.add_trace(
        go.Scatter(
            x=df_daily['date'],
            y=df_daily['conversion_rate'],
            name="Daily Conversion Rate",
            line=dict(color='lightblue', width=1),
            opacity=0.5
        ),
        secondary_y=False
    )
    
    # 7-day moving average
    fig.add_trace(
        go.Scatter(
            x=df_daily['date'],
            y=df_daily['7day_avg_conversion'],
            name="7-Day Average",
            line=dict(color='darkblue', width=3)
        ),
        secondary_y=False
    )
    
    # Signups volume
    fig.add_trace(
        go.Bar(
            x=df_daily['date'],
            y=df_daily['signups'],
            name="Daily Signups",
            marker_color='rgba(158,202,225,0.3)',
            yaxis='y2'
        ),
        secondary_y=True
    )
    
    fig.update_xaxes(title_text="Date")
    fig.update_yaxes(title_text="Conversion Rate (%)", secondary_y=False)
    fig.update_yaxes(title_text="Signups", secondary_y=True)
    
    fig.update_layout(
        title="Conversion Rate Trend",
        height=400,
        hovermode='x unified',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    return fig


def create_channel_performance_chart(df_channel):
    """Create channel performance comparison"""
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=df_channel['channel'],
        y=df_channel['conversion_rate'],
        text=df_channel['conversion_rate'].round(1),
        textposition='auto',
        marker_color=['#636EFA', '#EF553B', '#00CC96', '#AB63FA'],
        name='Conversion Rate'
    ))
    
    fig.update_layout(
        title="Conversion Rate by Channel",
        xaxis_title="Channel",
        yaxis_title="Conversion Rate (%)",
        height=350,
        showlegend=False
    )
    
    return fig


def create_user_activity_chart(df_daily):
    """Create user activity metrics over time"""
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Avg Sessions per User", "Avg Time Spent (min)"))
    
    fig.add_trace(
        go.Scatter(x=df_daily['date'], y=df_daily['avg_sessions'], 
                   fill='tozeroy', name='Sessions',
                   line=dict(color='#00CC96')),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=df_daily['date'], y=df_daily['avg_time_minutes'], 
                   fill='tozeroy', name='Time',
                   line=dict(color='#AB63FA')),
        row=1, col=2
    )
    
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=2)
    fig.update_layout(height=350, showlegend=False)
    
    return fig


# ============================================================================
# 4. CREATE DASH APP
# ============================================================================

app = Dash(__name__)

# Calculate key metrics for KPI cards
total_users = len(df_events)
total_purchases = df_events['stage_purchase'].sum()
overall_conversion = (total_purchases / total_users) * 100
total_revenue = df_events['purchase_value'].sum()
avg_order_value = total_revenue / total_purchases if total_purchases > 0 else 0

app.layout = html.Div([
    # Header
    html.Div([
        html.H1("🎯 Conversion & User Activity Dashboard", 
                style={'color': '#333', 'marginBottom': '10px'}),
        html.P("Real-time insights into user behavior and conversion performance",
               style={'color': '#666', 'fontSize': '14px'})
    ], style={'padding': '20px', 'backgroundColor': '#fff', 'borderBottom': '2px solid #eee'}),
    
    # KPI Cards Row
    html.Div([
        create_kpi_card("Total Users", f"{total_users:,}", "All time signups", "#636EFA"),
        create_kpi_card("Conversion Rate", f"{overall_conversion:.1f}%", "Overall CVR", "#00CC96"),
        create_kpi_card("Total Purchases", f"{int(total_purchases):,}", "Completed orders", "#EF553B"),
        create_kpi_card("Revenue", f"${total_revenue:,.0f}", f"AOV: ${avg_order_value:.2f}", "#AB63FA"),
    ], style={'display': 'flex', 'flexWrap': 'wrap', 'padding': '20px', 'gap': '10px'}),
    
    # Main Content
    html.Div([
        # Row 1: Funnel and Conversion Trend
        html.Div([
            html.Div([
                dcc.Graph(figure=create_funnel_chart(df_conversion))
            ], style={'flex': '1', 'minWidth': '400px'}),
            
            html.Div([
                dcc.Graph(figure=create_conversion_trend_chart(df_daily))
            ], style={'flex': '1.5', 'minWidth': '500px'}),
        ], style={'display': 'flex', 'flexWrap': 'wrap', 'gap': '20px', 'marginBottom': '20px'}),
        
        # Row 2: Channel Performance and Device Breakdown
        html.Div([
            html.Div([
                dcc.Graph(figure=create_channel_performance_chart(df_channel))
            ], style={'flex': '1', 'minWidth': '400px'}),
            
            html.Div([
                dcc.Graph(
                    figure=px.pie(df_device, values='users', names='device', 
                                 title='User Distribution by Device',
                                 color_discrete_sequence=px.colors.qualitative.Set3,
                                 height=350)
                )
            ], style={'flex': '1', 'minWidth': '400px'}),
        ], style={'display': 'flex', 'flexWrap': 'wrap', 'gap': '20px', 'marginBottom': '20px'}),
        
        # Row 3: User Activity Metrics
        html.Div([
            dcc.Graph(figure=create_user_activity_chart(df_daily))
        ]),
        
        # Detailed Table
        html.Div([
            html.H3("📋 Channel Performance Details", style={'color': '#333', 'marginTop': '20px'}),
            html.Div([
                html.Table([
                    html.Thead(
                        html.Tr([html.Th(col, style={'padding': '12px', 'textAlign': 'left', 
                                                      'backgroundColor': '#f8f9fa', 'borderBottom': '2px solid #dee2e6'}) 
                                for col in df_channel.columns])
                    ),
                    html.Tbody([
                        html.Tr([
                            html.Td(df_channel.iloc[i][col] if col not in ['conversion_rate', 'revenue'] 
                                   else (f"{df_channel.iloc[i][col]:.1f}%" if col == 'conversion_rate' 
                                        else f"${df_channel.iloc[i][col]:,.0f}"),
                                   style={'padding': '12px', 'borderBottom': '1px solid #dee2e6'})
                            for col in df_channel.columns
                        ]) for i in range(len(df_channel))
                    ])
                ], style={'width': '100%', 'borderCollapse': 'collapse', 'backgroundColor': '#fff'})
            ])
        ], style={'padding': '20px', 'backgroundColor': '#fff', 'borderRadius': '8px', 
                  'boxShadow': '0 2px 4px rgba(0,0,0,0.1)', 'marginTop': '20px'})
        
    ], style={'padding': '20px', 'backgroundColor': '#f5f5f5'}),
    
    # Footer
    html.Div([
        html.P("💡 Data refreshed: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
               style={'color': '#999', 'fontSize': '12px', 'textAlign': 'center'})
    ], style={'padding': '20px', 'backgroundColor': '#fff', 'borderTop': '1px solid #eee'})
    
], style={'fontFamily': 'Arial, sans-serif', 'backgroundColor': '#f5f5f5', 'minHeight': '100vh'})


# ============================================================================
# 5. RUN THE APP
# ============================================================================

if __name__ == '__main__':
    print("\n" + "="*70)
    print("🚀 Starting Dashboard Server...")
    print("="*70)
    print("\n📊 Dashboard will be available at: http://127.0.0.1:8050")
    print("\n💡 In Databricks, the dashboard will render inline in the notebook")
    print("="*70 + "\n")
    
    # For local development
    app.run(debug=True, port=8050)
    
    # For Databricks, use instead:
    # app.run_server(mode='inline', port=8050, dev_tools_ui=False)

📊 Generating sample data...
✓ Loaded 13,305 user records
✓ Date range: 2024-01-01 00:00:00 to 2024-03-30 00:00:00

🚀 Starting Dashboard Server...

📊 Dashboard will be available at: http://127.0.0.1:8050

💡 In Databricks, the dashboard will render inline in the notebook

